# Link Prediction in Condmat

In [1]:
from linkprediction import convert_to_set, filter_edgelist, get_distances, get_graph, giant_component, read_edges, report

edges = read_edges('out.bibsonomy-2ui')
print(f'Number of edges: {len(edges):.1e}')

Number of edges: 2.6e+06


In [2]:
edges_train_mature = filter_edgelist(edges, stop=50000)

1989-01-05 10:40:00
2004-10-24 20:12:05
no_selected_edges=50003 (2.0e-02)


In [3]:
edges_train_probe = filter_edgelist(edges, start=50000, stop=60000)

2004-10-24 20:12:06
2005-03-03 09:55:04
no_selected_edges=10006 (3.9e-03)


In [4]:
edges_test_mature = filter_edgelist(edges, stop=60000)

1989-01-05 10:40:00
2005-03-03 09:55:04
no_selected_edges=60009 (2.3e-02)


In [5]:
edges_test_probe = filter_edgelist(edges, start=60000, stop=70000)

2005-03-03 09:55:05
2005-06-20 00:51:54
no_selected_edges=9994 (3.9e-03)


## Set-up
Choose here the parameters on how you want to define the learn and assessing phase.

In [6]:
g_train_matured = giant_component(get_graph(edges_train_mature))
uv_train_probe = convert_to_set(edges_train_probe)

In [25]:
report(graph=g_train_matured, probes=uv_train_probe)

Number of probes: 3885
- already edge: 0 (0%)
- both nodes in graph: 47 (1%)
- not in graph: 3832 (99%)


In [7]:
g_test_matured = giant_component(get_graph(edges_test_mature))
uv_test_probe = convert_to_set(edges_test_probe)

In [27]:
report(graph=g_test_matured, probes=uv_test_probe)

Number of probes: 3074
- already edge: 0 (0%)
- both nodes in graph: 59 (2%)
- not in graph: 3005 (98%)


## Export

### Train

In [8]:
%%time
nodepairs_train, _ = get_distances(g_train_matured, cutoff=2)
targets_train = [nodepair in uv_train_probe for nodepair in tqdm(nodepairs_train)]


joblib.dump(nodepairs_train, 'temporal/train/2/nodepairs.pkl', protocol=5)
joblib.dump(targets_train, 'temporal/train/2/targets.pkl', protocol=5)
joblib.dump(g_train_matured, 'temporal/train/2/graph.pkl', protocol=5)

get_distances:  10%|█         | 1086/10776 [01:11<10:34, 15.27it/s]


KeyboardInterrupt: 

In [ ]:
print(f'{sum(targets_train) / len(nodepairs_train):e}')

### Test

In [ ]:
%%time
nodepairs_test, _ = get_distances(g_test_matured, cutoff=2)
targets_test = [nodepair in uv_test_probe for nodepair in tqdm(nodepairs_test)]

In [ ]:
%%time
joblib.dump(nodepairs_test, 'temporal/test/2/nodepairs.pkl', protocol=5)
joblib.dump(targets_test, 'temporal/test/2/targets.pkl', protocol=5)
joblib.dump(g_test_matured, 'temporal/test/2/graph.pkl', protocol=5)

In [ ]:
print(f'{sum(targets_test) / len(nodepairs_test):e}')

## Hyperparameter selection

### XGBoost

$n=2$

In [24]:
def get_x_y(df: pd.DataFrame): return df.drop(columns='target').values, df['target'].values
def gridsearch(df: pd.DataFrame, random_state=1, also_random=True, max_depth=[1, 2]) -> pd.DataFrame:
  X, y = get_x_y(df)
  
  
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(random_state=random_state, tree_method='hist', n_jobs=6)
  gridsearch = GridSearchCV(
    clf, 
    param_grid=dict(max_depth=max_depth, scale_pos_weight=[sum(~y_train)/sum(y_train), 1]), 
    scoring='average_precision', 
    n_jobs=30,
    cv=StratifiedKFold(shuffle=True, random_state=random_state),
    return_train_score=True
  )
  
  if also_random: 
    gridsearch_random = copy.deepcopy(gridsearch)
    np.random.seed(random_state)
    y_random = copy.deepcopy(y_train)
    np.random.shuffle(y_random)
  
  gridsearch.fit(X_train, y_train)
  df_dict = dict(
      mean_train=gridsearch.cv_results_['mean_train_score'],
      std_train=gridsearch.cv_results_['std_train_score'],
      mean_val=gridsearch.cv_results_['mean_test_score'],
      std_val=gridsearch.cv_results_['std_test_score'],
      val_fold0=gridsearch.cv_results_[f'split0_test_score'],
      val_fold1=gridsearch.cv_results_[f'split1_test_score'],
      val_fold2=gridsearch.cv_results_[f'split2_test_score'],
      val_fold3=gridsearch.cv_results_[f'split3_test_score'],
      val_fold4=gridsearch.cv_results_[f'split4_test_score']
  )
  
  if also_random: 
    gridsearch_random.fit(X_train, y_random)
    df_dict['mean_train_random']=gridsearch_random.cv_results_['mean_train_score']
    df_dict['std_train_random']=gridsearch_random.cv_results_['std_train_score']
    df_dict['mean_val_random']=gridsearch_random.cv_results_['mean_test_score']
    df_dict['std_val_random']=gridsearch_random.cv_results_['std_test_score']
  df = pd.DataFrame(df_dict, index=pd.Index([(d['max_depth'], d['scale_pos_weight'] > 1) for d in gridsearch.cv_results_['params']], name=('max_depth', 'balanced')))
  df['diff_train_val'] = df['mean_val'] - df['mean_train']
  df['rstd_test'] = df['std_val'] / df['mean_val']
  if also_random: df['val_over_random'] = df['mean_val'] - df['mean_val_random']
  return df.sort_values('mean_val', ascending=False)
    
def report_performance(df_train: pd.DataFrame, df_test: pd.DataFrame, random_state=1, max_depth=1, tree_method='hist', balanced=True, n_jobs=128):
  X, y = get_x_y(df_train)
  clf = XGBClassifier(max_depth=max_depth, n_jobs=128, tree_method=tree_method, scale_pos_weight=sum(~y)/sum(y) if balanced else 1 , random_state=random_state)
  clf.fit(X, y)
  X_test, y_test = get_x_y(df_test)
  y_pred = clf.predict_proba(X_test)[:,1]
  return average_precision_score(y_test, y_pred), roc_auc_score(y_test, y_pred)

In [ ]:
hps2 = gridsearch(pd.read_pickle(f'temporal/train/2/features.pkl'))

In [ ]:
hps2[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [ ]:
report_performance(df_train=pd.read_pickle(f'temp/b1/train/2/features.pkl'), df_test=pd.read_pickle(f'temp/b1/test/2/features.pkl'), balanced=False)